# Aufgabe Stream

Wir lesen zuerst die Datei ein. Der Befehl stream["ZNGROUP"] = stream["ZNGROUP"].apply(str) ist notwendig, dass pandas die Zahlen in der Spalte ZNGROUP als Faktorstufen 1-4 und nicht als Zahlen 1-4 interpretiert.

In [2]:
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
import pandas as pd
from patsy.contrasts import Sum
stream = pd.read_csv('stream.dat', sep=' ', header=0)
stream["ZNGROUP"] = stream["ZNGROUP"].apply(str)

stream.head()

,STREAM,ZINC,DIVERSITY,ZNGROUP
0,Eagle,BACK,2.27,1
1,Eagle,HIGH,1.25,4
2,Eagle,HIGH,1.15,4
3,Eagle,MED,1.62,3
4,Blue,BACK,1.70,1


Wir berechnen nun die Mittelwerte für die Zinkgruppen:

In [3]:
stream.groupby("ZNGROUP").mean()

,DIVERSITY
ZNGROUP,
1,1.797500
2,2.032500
3,1.717778
4,1.277778


Diese Gruppenmittelwerte entsprechen:
$$
\hat{\mu}_1=1.7975,
\qquad 
\hat{\mu}_2=2.0325,
\qquad 
\hat{\mu}_3=1.7178,
\qquad 
\hat{\mu}_4=1.2778
$$
Die $\hat{\ }$-Variante wird hier gewählt, da dies aus Daten gemessene Werte sind. Ohne Hut sind das theoretische Werte, die allgemein nicht bekannt sind.

Wir interessieren uns für die _Unterschiede_ zwischen den Mittelwerten und da ist obige Schreibweise nicht so praktisch. Aus diesem Grund wählt man (es gibt viele verschiedene Varianten) das sogenannte Gruppenmittelmodell
$$
\mu_i
=\mu+\alpha_i+\epsilon
$$
wobei $\mu$ etwas salopp gesagt, die Baseline ist. Diese kann wieder beliebig gewählt werden, die $\alpha$'s lauten dann halt entsprechen anders. Damit man aber die _Unterschiede_ berücksichtigen möchte, gibt _eine_ Einschrängkung.

Beispielsweise wird das Gruppenmittel $\mu_1$ als Baseline gewählt, also
$$
\mu_1=\mu
$$
Somit ist $\alpha_1=0$ und dies ist eine sogenannte Randbedingung. Ein Wert können wir _nicht_ beliebig wählen, sondern ist schon vorgegeben (siehe auch Folie 20, Block 9). Alle anderen Mittelwerte werden mit dieser Baseline verglichen. 

Das ist das, was statsmodels im folgenden Code bestimmt:

In [4]:
fit = ols("DIVERSITY ~ ZNGROUP", data=stream).fit()
fit.params

Intercept       1.797500
ZNGROUP[T.2]    0.235000
ZNGROUP[T.3]   -0.079722
ZNGROUP[T.4]   -0.519722
dtype: float64

Es gilt dann:
$$
\hat{\mu}_1=1.7975,
\qquad 
\hat{\mu}_2=1.7975+0.235=2.03255,
\qquad 
\hat{\mu}_3=1.7975-0.0797=1.7178,
\qquad 
\hat{\mu}_4=1.7975-0.5197=1.2778
$$
Man beachte, es werden nur 3 $\alpha$'s ausgegeben, da das 4. klar ist. 

Nun ist $\alpha_1=0$ eine Möglichkeit, die Randbedingung zu wählen. Wir hätten natürlich genauso gut $\alpha_3=0$ wählen können. Die ausgegebenen Werte oben sind dann zwar verschieden, aber die daraus berechneten Mittelwerte sind gleich. 

Wir können aber auch folgende Randbedingung wählen
$$
\alpha_1+\alpha_2+\alpha_3+\alpha_4
=0
$$
Dies wird dann mit folgendem Code erreicht

In [5]:
fit = ols("DIVERSITY ~ C(ZNGROUP, Sum)", data=stream).fit()
fit.params

Intercept               1.706389
C(ZNGROUP, Sum)[S.1]    0.091111
C(ZNGROUP, Sum)[S.2]    0.326111
C(ZNGROUP, Sum)[S.3]    0.011389
dtype: float64

Hier sehen die Werte anders aus als im Output weiter oben, da die Randbedingung eine andere ist. Aber die berechneten Mittelwerte bleiben gleich:
$$
\hat{\mu}_1=1.7064+0.0911=1.7975,
\qquad 
\hat{\mu}_2=1.7064+0.3261=2.03255,
\qquad 
\hat{\mu}_3=1.7064+0.0.0114=1.7178,
\qquad 
\hat{\mu}_4=1.7064+(-0.0911-0.3261-0.01140)=1.2778
$$
Der letzte Wert kommt aus der Randbedingung $\alpha_1+\alpha_2+\alpha_3+\alpha_4=0$ oder 
$$
\alpha_4=-\alpha_1-\alpha_2-\alpha_3
$$
Auch hier sind nur wieder 3 $\alpha$'s angegeben, da das 4 aus den anderen 3 folgt.

Es ist nicht so, dass die Randbedingung nicht wichtig ist, sie ist sogar zentral für die Berechnungen des $F$-Wertes, aber es spielt keine Rolle _welche_ Randbedingung wir wählen. Die berechneten Mittelwerte müssen gleich bleiben. 

Nun bleibt halt noch die Frage, welche Randbedingung gewählt wird. Das hat oft auch historische Gründe, wo die Randbedingung so gewählt wurde, dass die Berechnungen der $p$-Werte möglichst einfach wurde. Das ist mit den heutigen Hilfsmittel keine Thema mehr.  

Oder aus Gründen für die Interpretation. Im Beispiel der Diäten, möchte der Hersteller der wirksamsten Diät Werbung machen und wählt dann die eigene Diät als Baseline und zeigt, dass die anderen beiden schlechter sind. 